In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
# import shap
# import seaborn as sns
from catboost import *
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc
#%matplotlib
###########import packages##########
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import lightgbm
import catboost
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.impute import SimpleImputer
import keras_metrics as km
from keras.callbacks import EarlyStopping 
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from scipy import interp
from itertools import cycle
early_stopping=keras.callbacks.EarlyStopping(
 monitor="val_loss", 
 patience=20, 
 verbose=0, 
 mode="auto"
)
from sklearn.metrics import r2_score
#%matplotlib
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=np.sqrt(sum(squaredError)/len(squaredError))
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})
def intergate(y_pred):
    length=y_pred.shape[0]
    print(length)
    for i in range (0,length):
        if y_pred[i][0]>=0.5:
            y_pred[i][0]=1
        else:
            y_pred[i][0]=0
    return y_pred
accuracy={}
###########loading data##########
fdata=pd.read_csv('database_filled_CD.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'Operating Temperature (℃)',#0
                      'Operating Pressure (bar)',#1
                      'Flow Rate (mL min-1)',#2
                      'Active Area (cm2)', #3
                      'Ir wt. %',#4
                      'Ru wt.%',#5
                      'O wt. %',#6
                      'C wt. %',#7
                      'Pure_0/Supported_1',#8
                      'I/C in Anode',#9
                      'Pt wt. %',#10
                      'I/C in Cathode',#11  
                      'Anode Precious Metal Loading (mg cm-2 Ir/Ru/Pt/Pd)',#12
                      'Cathode Precious Metal Loading (mg cm-2 Pt/Pd)',#13
                      'CCM_0/GDE_1',#14    
                      'Membrane Thickness (μm)',#15
                      'EW',#16
                      'Minimum Stability Current Density (A cm-2)',#17
                      'Maximum Stability Current Density (A cm-2)',#18
                      'Fluctuation period (h)',#19
                      'Stability Test Time (h-1)'
]]
###########data standardization##########
standardized_data = (raw_data-np.mean(raw_data,axis=0))/np.std(raw_data,axis=0)
###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input=standardized_data.iloc[:,0:21]
raw_input_global=raw_data.iloc[:,0:21]
print('ready')

In [ ]:
CD19=fdata.loc[:,['Current at 1.9 V']].values

In [ ]:
def summarize(list1):
    total=0
    for ele in range(0, len(list1)):
        total = total + list1[ele]
    return total

In [ ]:
pca_3 = PCA(n_components=3)
chem_3 = pca_3.fit_transform(raw_data)     
pca_2 = PCA(n_components=2)
chem_2 = pca_2.fit_transform(raw_data)

In [ ]:
seed=8
###########train test splitting##########
X_train, X_test, y_train, y_test = train_test_split(chem_3, CD19, test_size=.15,random_state=seed)

In [ ]:
for batch_size_number in [16,24,32]:
    for reg in [0,0.0001,0.001]:
        for dropout_rate in [0,0.1,0.2,0.3,0.4,0.5]:
            for neurons1 in range(200,1000,100):
                for epochs_number in range(150,850,100):
                    for learning_rate_search in [0.0005,0.00075,0.001,0.0025,0.005,0.0075,0.01,0.02]:
                        for activation1 in ['relu']:
                            regularizer=keras.regularizers.l2(reg)
                            ###########keras ANN model construction##########
                            model = Sequential() 
                            model.add(Dense(neurons1, input_dim=3, kernel_initializer='random_normal',
                                            bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                            model.add(Dropout(dropout_rate))
                            model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                            bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                            model.add(Dropout(dropout_rate))
                            model.add(Dense(1, input_dim=neurons1, activation='linear'))
                            adam=optimizers.Adam(lr=learning_rate_search)
                            model.compile(loss='mse', optimizer=adam)
                            print('training...')
                            model.fit(X_train, y_train,verbose=0, batch_size=batch_size_number,epochs=epochs_number,validation_split=0.2,callbacks=[early_stopping])
                            result=model.predict(X_test)
                            result_train=model.predict(X_train)
                            ###########get RMSE and R2 on the test set##########
                            x_prediction_07=result[:,0]
                            y_real_07=y_test[:,0]
                            x_prediction_07_series=pd.Series(x_prediction_07)
                            y_real_07_series=pd.Series(y_real_07)
                            #training set
                            x_prediction_07_train=result_train[:,0]
                            y_real_07_train=y_train[:,0]
                            x_prediction_07_series_train=pd.Series(x_prediction_07_train)
                            y_real_07_series_train=pd.Series(y_real_07_train)
                            ###########evaluating the regression quality##########
                            corr_ann = round(x_prediction_07_series.corr(y_real_07_series), 5)
                            error_val= compute_mae_mse_rmse(x_prediction_07,y_real_07)
                            corr_ann_train = round(x_prediction_07_series_train.corr(y_real_07_series_train), 5)
                            error_val_train= compute_mae_mse_rmse(x_prediction_07_train,y_real_07_train)
                            print('TEST SET scatter corr',corr_ann,'scatter error',error_val,'TEST R2',error_val[3])
                            print('TRAINING SET scatter corr',corr_ann_train,'scatter error',error_val_train,'R2',error_val_train[3])
                            print(neurons1,epochs_number,learning_rate_search,dropout_rate,batch_size_number,reg,activation1)                          
                            x_y_x=np.arange(0,6.5,0.1)
                            x_y_y=np.arange(0,6.5,0.1)
                            fig = plt.figure()
                            ax = fig.add_subplot(111)
                            ax.scatter(x_prediction_07,y_real_07,color='red',label='Artificial Neural Network Test Set',alpha=0.75)
                            ax.scatter(x_prediction_07_train,y_real_07_train,color='blue',label='Artificial Neural Network Training Set',alpha=0.25,marker="^")
                            ax.plot(x_y_x,x_y_y)
                            plt.legend()
                            plt.xlabel(u"Predicted_Current_Density (mA cm^-2)@1.9V (vs. RHE)")
                            plt.ylabel(u"Real_Current_Density (mA cm^-2)@1.9V (vs. RHE)")
                            plt.savefig('CD19 ANN PCA 3D %s %s %s %s %s %s %s.png' %(neurons1,epochs_number,learning_rate_search,dropout_rate,batch_size_number,reg,activation1))
                            K.clear_session()
print(accuracy)